In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

import argparse
import os
import sys
import random
import warnings
import cv2

import numpy as np

import matplotlib.pyplot as plt

from datetime import datetime

current_time = datetime.now().strftime('%H%M_%m%d%Y')

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14317013328114034532
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11131259456
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3781685903256362717
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
# Set seed value for all of frameworks
seed_value= 2021 
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

In [4]:
# Call Loss Functions
from loss_functions import dice_coef_loss
# Call Data Generator
from data_generator import data_generator
# Call U-Net Model
from unet_models import unetModel_basic_4

Using TensorFlow backend.


In [4]:
# variables
# dataFolder = '/sbgenomics/project-files/DATASET_BUD/'
# EPOCH = 1
# BS = 16
# model_no = 1
# cuda_no = 0
# model_name = 'test'
# size_img = 512
# scale_factor = 1

parameters_dict = {
    "data_folder": '/sbgenomics/project-files/DATASET_BUD/',
    "batch_size": 16,
    "epoch": 100,
    "model_no": 1,
    "gpu_no": '0',
    "model_name": 'test',
    "learning_rate": 0.001,
    "size_img": 512,
    "scale_factor": 1
}

os.environ["CUDA_VISIBLE_DEVICES"] = parameters_dict['gpu_no']

# Set Main Parameters
IMG_WIDTH = parameters_dict['size_img']
IMG_HEIGHT = parameters_dict['size_img']
IMG_CHANNELS = 3
TRAIN_PATH = parameters_dict['data_folder']

In [17]:
# Write Directory
dir_write = 'OUTPUT/RUN_TRAIN/' + '' + '/Run_Train_' + 'test' + '_' + str(current_time) + '/'
dir_pred = dir_write + 'Pred_imgs/'
dir_model = dir_write + 'Model/'
dir_log = dir_write + 'Log/'
if not os.path.exists(dir_write):
    os.makedirs(dir_write)
    os.makedirs(dir_pred)
    os.makedirs(dir_model)
    os.makedirs(dir_log)

In [18]:
naming_parameter = '_'.join(list([key + "-" + str(val) for key,val in parameters_dict.items()])[1:]) 
parameters_dict['model_name'] = naming_parameter
naming_parameter

'batch_size-16_epoch-100_model_no-1_cuda_no-0_model_name-test_learning_rate-0.001_size_img-512_scale_factor-1'

In [19]:
# Write hyperparameters to txt file

fname_hyper = dir_write + 'parameters_' + parameters_dict['model_name'] + '_' + str(current_time) + '.txt'
str_info = 'dataset: ' + parameters_dict['data_folder'] + '\n' + 'EPOCH: ' + str(parameters_dict['epoch']) +'\n' + \
            'BS: ' + str(parameters_dict['batch_size']) + '\n' + 'model no: ' + str(parameters_dict['model_no']) + '\n' + \
            'model name: ' + parameters_dict['model_name'] + '\n' + 'size of img: ' + str(parameters_dict['size_img']) + '\n' + \
            'weight scale: ' + str(parameters_dict['scale_factor'])
file_info = open(fname_hyper, 'w')
file_info.write(str_info)
file_info.close()

# Data

In [9]:
print(f'[DEBUG] Getting and resizing({IMG_WIDTH}x{IMG_HEIGHT}) train images and masks before ... ')

# Get and resize train images and masks
train_cpt = int(sum([len(files) for r, d, files in os.walk(TRAIN_PATH + "img/")]))

X_train = np.ndarray((train_cpt, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.float32)
Y_train = np.ndarray((train_cpt, IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.float32)  # dtype=np.bool)

print(f'[DEBUG] Getting and Resizing Train Images and Masks after... ')
sys.stdout.flush()

path = TRAIN_PATH + "img/"
pathMask = TRAIN_PATH + "mask/"
print('Path to img: {}'.format(path))

_, _, filesInPath = next(os.walk(path))
_, _, filesInPathMask = next(os.walk(pathMask))

print(f'# of Samples Image: {len(filesInPath)}\t# of samples mask: {len(filesInPathMask)}')

filesInPath = sorted(filesInPath)
filesInPathMask = sorted(filesInPathMask)

for i, f in enumerate(filesInPath):
    img = cv2.imread(path + f)
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
    img = img / 255
    X_train[i] = img

for i, fm in enumerate(filesInPathMask):
    img_mask = cv2.imread(pathMask + fm, cv2.IMREAD_GRAYSCALE)
    img_mask = cv2.resize(img_mask, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
    img_mask = img_mask / 255
    img_mask = np.expand_dims(img_mask, axis=-1)
    Y_train[i] = img_mask

print(f"[DEBUG][INFO] Data Matrix: {round(X_train.nbytes / (1024 * 1000.0),3)} mb")
###########################
pixels = Y_train.flatten().reshape(train_cpt, IMG_HEIGHT*IMG_WIDTH)

weights_train = pixels.copy()

pixels = np.expand_dims(pixels, axis = -1)

[DEBUG] Getting and resizing(512x512) train images and masks before ... 
[DEBUG] Getting and Resizing Train Images and Masks after... 
Path to img: /sbgenomics/project-files/DATASET_BUD/img/
# of Samples Image: 254	# of samples mask: 254
[DEBUG][INFO] Data Matrix: 780.288 mb


# Model

In [20]:
model = unetModel_basic_4(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, lr_rate=parameters_dict['learning_rate'])

In [21]:
# Model Summary
# print(model.summary())
file_info = open(fname_hyper, 'a')
model.summary(print_fn=lambda x: file_info.write('\n' + x + '\n'))
file_info.close()

In [22]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

# Stop training when a monitoring quantity has stopped improving
# earlystopper = EarlyStopping(monitor='val_loss', patience=100, verbose=1)

# Save the model after every epoch
checkpointer = ModelCheckpoint(parameters_dict['model_name'] + '_main_modelCheckpoint.h5', verbose=0, monitor='val_loss', \
                               save_best_only=True, save_weights_only=False, period=1, mode='auto')

name_unique_ext = parameters_dict['model_name']
csv_logger = CSVLogger('{}/log_{}.training.csv'.format(dir_log, name_unique_ext))
reduce_lr = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, patience=10, verbose=1, mode='max', cooldown=1, min_lr=0.000001)

## Fit Model

In [23]:
# Free up RAM in case the model definition cells were run multiple times
from tensorflow.keras import backend as K
K.clear_session()

In [24]:
# Fit model
print("[INFO] Model fitting...")
results = model.fit(X_train, pixels, validation_split=0.2, batch_size=parameters_dict['batch_size'], epochs=parameters_dict['epoch'],
                    callbacks=[checkpointer, csv_logger, reduce_lr], verbose=1, shuffle=True)#, sample_weight=weights_train)
print("[INFO] Model fitting is done!")

[INFO] Model fitting...
Epoch 1/100
13/13 [==============================] - 20s 1s/step - loss: 0.9734 - dice_coef: 0.0268 - val_loss: 0.9065 - val_dice_coef: 0.1190
Epoch 2/100
13/13 [==============================] - 17s 1s/step - loss: 0.9303 - dice_coef: 0.0697 - val_loss: 0.8963 - val_dice_coef: 0.1204
Epoch 3/100
13/13 [==============================] - 17s 1s/step - loss: 0.9250 - dice_coef: 0.0749 - val_loss: 0.8857 - val_dice_coef: 0.1398
Epoch 4/100
13/13 [==============================] - 17s 1s/step - loss: 0.9168 - dice_coef: 0.0831 - val_loss: 0.8439 - val_dice_coef: 0.1800
Epoch 5/100
13/13 [==============================] - 17s 1s/step - loss: 0.9013 - dice_coef: 0.0987 - val_loss: 0.7841 - val_dice_coef: 0.2434
Epoch 6/100
13/13 [==============================] - 17s 1s/step - loss: 0.8396 - dice_coef: 0.1610 - val_loss: 0.8074 - val_dice_coef: 0.2447
Epoch 7/100
13/13 [==============================] - 17s 1s/step - loss: 0.7804 - dice_coef: 0.2200 - val_loss: 0.7057

In [25]:
# Write model history to the file
import pandas as pd
pd.DataFrame(results.history).to_csv(dir_write + "history_" + parameters_dict['model_name'] + ".csv")

In [28]:
# serialize model to JSON
model_json = model.to_json()
with open( parameters_dict['model_name'] + ".json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights( parameters_dict['model_name'] + ".h5")
print("Saved model: {} to disk".format(parameters_dict['model_name']))

Saved model: batch_size-16_epoch-100_model_no-1_cuda_no-0_model_name-test_learning_rate-0.001_size_img-512_scale_factor-1 to disk


In [30]:
# Prediction
print('Prediction....')
preds_test = model.predict(X_train[:10], batch_size= parameters_dict['batch_size'], verbose=2)

Prediction....
1/1 - 3s


In [31]:
preds_test.shape

(10, 262144, 1)

In [32]:
preds_reshaped = np.ndarray((len(preds_test), IMG_HEIGHT, IMG_WIDTH), dtype=np.float32)
for i in range(len(preds_test)):
    preds_reshaped[i] = preds_test[i].reshape(IMG_HEIGHT, IMG_WIDTH)

preds_upsampled = []
for i in range(len(preds_test)):
    preds_upsampled.append(np.expand_dims(cv2.resize(preds_reshaped[i], (IMG_HEIGHT, IMG_WIDTH)), axis=-1))
print("[INFO] Upsampling is done!(upsampled to ({}, {}) from ({}, {})".format(IMG_HEIGHT, IMG_WIDTH, preds_test[i].shape[0], preds_test[i].shape[1]))

[INFO] Upsampling is done!(upsampled to (512, 512) from (262144, 1)


In [33]:
for k in range(10):
    img_mask = np.zeros((IMG_HEIGHT, IMG_WIDTH))
    img = preds_upsampled[k]

    for i in range(len(img)):
        for j in range(len(img[i])):
            if img[i][j] > 0.25:
                img_mask[i][j] = 1
            else:
                img_mask[i][j] = 0
    img_mask *= 255

    cv2.imwrite(dir_pred + filesInPath[k], img_mask)
print('FINISHED TRAINING!')

FINISHED TRAINING!


In [35]:
preds_reshaped

array([[[1.56420190e-02, 1.04882289e-03, 8.24602757e-05, ...,
         1.02202735e-04, 4.24940488e-04, 6.65157381e-03],
        [1.30318440e-04, 1.07092535e-06, 8.74409452e-08, ...,
         4.73958636e-07, 5.10872269e-06, 4.17713367e-04],
        [2.16416229e-05, 2.05903135e-07, 3.97768041e-08, ...,
         3.46473001e-07, 3.51130984e-06, 3.10111354e-04],
        ...,
        [4.03091944e-05, 9.14032299e-08, 7.50734408e-09, ...,
         1.40608336e-09, 5.63318849e-08, 3.44579676e-05],
        [1.60592972e-04, 1.23540588e-06, 2.05338978e-07, ...,
         2.82650578e-08, 5.73341765e-07, 1.31053413e-04],
        [6.55167131e-03, 2.13950363e-04, 5.08847843e-05, ...,
         1.74235686e-06, 1.04318806e-05, 5.40183799e-04]],

       [[1.03768837e-02, 5.48331591e-04, 3.83087827e-05, ...,
         1.24171711e-04, 5.00192342e-04, 7.39447959e-03],
        [5.53901664e-05, 4.13871817e-07, 3.65674460e-08, ...,
         5.93760547e-07, 6.15330464e-06, 4.88606165e-04],
        [1.17744776e-05, 